# Test anchor detection on FB data
Let's try to detect anchors using the basic strategies:

- CITY, MUNICPALITY
- LOC + DESCRIPTOR + ANCHOR

In [53]:
import pandas as pd
import numpy as np
from ast import literal_eval

In [54]:
# load data
data = pd.read_csv('../../data/facebook-maria/combined_group_data_es_tagged_valid.tsv', sep='\t', index_col=False, converters={'status_message_tags' : literal_eval})
data = data.assign(**{'valid_loc' : data.loc[:, ['LOC_GN', 'LOC_OSM']].max(axis=1)})
data_valid = data[data.loc[:, 'valid_loc']==1]
print('%d/%d valid NEs'%(data_valid.shape[0], data.shape[0]))
display(data.head())

9389/25435 valid NEs


,group_id,status_author_id,status_message,status_lang,status_id,status_published,status_message_clean,status_message_tags,status_message_tags_ne,status_len,status_message_tags_ne_LOC,LOC_GN,LOC_OSM,NE,valid_loc
0,351272391991842,10159623474155515,Estoy preocupada por mi tia vicky vazquez que ...,es,351272391991842_362333787552369,2017-10-19 18:25:23,Estoy preocupada por mi tia vicky vazquez que ...,"[(Estoy, O), (preocupada, O), (por, O), (mi, O...","[('asma', 'CAUSE_OF_DEATH'), ('Dorado', 'ORGAN...",579,['Dorado'],1,1,Dorado,1
1,351272391991842,1716175958400853,En apoyo al alcalde Carlos Lopez y al municipi...,es,351272391991842_361453717640376,2017-10-17 17:38:08,En apoyo al alcalde Carlos Lopez y al municipi...,"[(En, O), (apoyo, O), (a, O), (el, O), (alcald...","[('alcalde', 'TITLE'), ('Carlos Lopez', 'PERSO...",74,['Dorado Dorado'],0,0,Dorado Dorado,0
2,351272391991842,141039043205101,Pueblo de Puerto Rico no se monten en las gran...,es,351272391991842_358929311226150,2017-10-11 14:14:22,Pueblo de Puerto Rico no se monten en las gran...,"[(Pueblo, O), (de, O), (Puerto, COUNTRY), (Ric...","[('Puerto Rico', 'COUNTRY'), ('America', 'MISC...",356,['shelter'],0,1,shelter,1
3,351272391991842,2060154724214607,"Guayama, Dorado, y Piñones",es,351272391991842_357797888005959,2017-10-08 14:52:49,"Guayama, Dorado, y Piñones","[(Guayama, LOCATION), (,, O), (Dorado, O), (,,...","[('Guayama', 'LOCATION')]",26,['Guayama'],1,1,Guayama,1
4,351272391991842,10155779649437154,"Estoy buscando mi tío, Ivan J. Porrata. Su her...",es,351272391991842_355121381606943,2017-10-01 21:46:29,"Estoy buscando mi tío, Ivan J. Porrata. Su her...","[(Estoy, O), (buscando, O), (mi, O), (tío, O),...","[('Ivan J. Porrata', 'PERSON'), ('Tulio', 'PER...",309,"['Utuado', 'Toa', 'Baja', 'P.O. Box', 'Dorado'...",1,1,Utuado,1


In [7]:
import re
NE_counts = data_valid.loc[:, 'NE'].value_counts()
display(NE_counts)
city_matcher = re.compile('barrio')
street_matcher = re.compile('calle')
# neighborhood counts
display(NE_counts.loc[[x for x in NE_counts.index if city_matcher.search(x.lower()) is not None]])
# street counts
display(NE_counts.loc[[x for x in NE_counts.index if street_matcher.search(x.lower()) is not None]])

Guayama                      712
Coamo                        419
Barranquitas                 410
Quebradillas                 289
Yabucoa                      258
Corozal                      211
Cayey                        200
Cidra                        193
Utuado                       139
Walmart                      129
San Juan                     127
Ponce                        119
Lajas                        110
Patillas                      99
Isabela                       84
San Antonio                   84
Caguas                        77
Naranjito                     77
Barrancas                     74
Arroyo                        65
Sabana                        57
el Pueblo                     54
Llanos                        53
Palmarejo                     53
Bayamon                       51
Helechal                      51
el Carmen                     45
Aguadilla                     44
Mayaguez                      42
USPS                          40
          

Barrio Limones         9
Barrio Nuevo           3
Quebradillas Barrio    3
Barrio Yaurel          3
Barrio Pueblo          3
Helechal Barrio        1
quebradillas barrio    1
Barrio Santa Ana       1
Barrio Belgica         1
Name: NE, dtype: int64

Calle                      33
Calle Abajo                 4
Calle Barcelona             2
Calle 9                     2
Calle I                     2
Calle Union                 2
Callejón Bravo              2
Calle Baldorioty            2
Calle Crisantemo            2
Cuatro Calles               2
Calle Morse                 2
Calle Cruz Torres           1
Calle Almagro               1
Calle Orquídea              1
Calle Alturas               1
Calle Grand Stand           1
Calle Sagitario             1
Calle Teodomiro Ramírez     1
Calle Guaraguao             1
Calle San Judas             1
Calle Domenech              1
Calle San Jose              1
Calle Unión                 1
calle                       1
Calle Opal                  1
Calle Luna                  1
Calle Esmeralda             1
Calle Violetas              1
Calle Comercio              1
Calle José                  1
Calle Flamboyan             1
Calle Ruiz Belvis           1
Calle Aleli                 1
Calle Vill

Not as many cities/streets as I thought! This means we'll have to try using everything...yikes.

## Test pattern matching

Let's set up the pattern matcher for the CITY, MUNICIPALITY pattern. This requires a list of all municipalities which we can get from the gazetteer.

In [8]:
geonames_data = pd.read_csv('/hg190/corpora/GeoNames/allCountriesSimplified.tsv', sep='\t', index_col=False)
geonames_data_PR = geonames_data[geonames_data.loc[:, 'country']=='PR']
geonames_data_PR_muni = geonames_data_PR[geonames_data_PR.loc[:, 'feature_code']=='ADM1']

In [21]:
data_valid.iloc[300, :].loc[['status_message', 'NE']].values

array(['Mi gente a pesar de tanta desgracia en nuestra isla ya gracias a Dios muchas personas de Corozal se han podido comunicar con sus Familiares desde Río hondo, Cataño, por los filtros en fin estamos bien contentos de poder ayudarnos unos a los otros por esta página y por Zello todos los que estamos fuera de PR estamos recopilando información que nos dan nuestros Familiares y se las estamos haciendo llegar lo hacemos con mucho cariño esto ha hecho que estemos muchas personas unidas de distintas partes de los Estados Unidos mi gente la unión hace la fuerza!!!!!!! 🇵🇷♥️🇵🇷♥️🇵🇷si reciben una llamada de un número desconocido contesten por favor ya están pasando por los distintos barrios con teléfonos civiles!para que los familiares tengan contacto con nosotros que estamos fuera de PR',
       'Río hondo'], dtype=object)

In [36]:
import re
from unidecode import unidecode
def clean_txt(x):
    return unidecode(x.lower())
PR_muni_names = [clean_txt(x.replace(' Municipio', '')) for x in geonames_data_PR_muni.loc[:, 'name'].unique()]

def match_city_loc_pattern(NE, txt, loc_names):
    NE_matcher = re.compile('|'.join(['%s,?\s?%s|%s,?\s?%s'%(clean_txt(NE), x, x, clean_txt(NE)) for x in loc_names]))
    city_loc_match = NE_matcher.search(clean_txt(txt))
    return city_loc_match is not None
NE_test = 'Río hondo'
txt_test = """
Mi gente a pesar de tanta desgracia en nuestra isla ya gracias a Dios muchas personas de Corozal se han podido comunicar con sus Familiares desde Río hondo, Cataño, por los filtros en fin estamos bien contentos de poder ayudarnos unos a los otros por esta página y por Zello todos los que estamos fuera de PR estamos recopilando información que nos dan nuestros Familiares y se las estamos haciendo llegar lo hacemos con mucho cariño esto ha hecho que estemos muchas personas unidas de distintas partes de los Estados Unidos mi gente la unión hace la fuerza!!!!!!! 🇵🇷♥️🇵🇷♥️🇵🇷si reciben una llamada de un número desconocido contesten por favor ya están pasando por los distintos barrios con teléfonos civiles!para que los familiares tengan contacto con nosotros que estamos fuera de PR
"""
print(match_city_loc_pattern(NE_test, txt_test, PR_muni_names))

True


In [125]:
print(','.join(['"%s"'%(x.replace(' Municipio', '')) for x in geonames_data_PR_muni.loc[:, 'name'].unique()]))

"Adjuntas","Aguada","Aguadilla","Aguas Buenas","Aibonito","Añasco","Arecibo","Arroyo","Barceloneta","Barranquitas","Bayamón","Cabo Rojo","Caguas","Camuy","Canóvanas","Carolina","Cataño","Cayey","Ceiba","Ciales","Cidra","Coamo","Comerío","Corozal","Culebra","Dorado","Fajardo","Florida","Guánica","Guayama","Guayanilla","Guaynabo","Gurabo","Hatillo","Hormigueros","Humacao","Municipio de Isabela","Municipio de Jayuya","Juana Díaz","Municipio de Juncos","Lajas","Lares","Las Marías","Las Piedras","Loíza","Luquillo","Manatí","Maricao","Maunabo","Mayagüez","Moca","Morovis","Naguabo","Naranjito","Orocovis","Patillas","Peñuelas","Ponce","Rincón","Quebradillas","Río Grande","Sabana Grande","Salinas","San Germán","San Juan","San Lorenzo","San Sebastián","Santa Isabel","Toa Alta","Toa Baja","Trujillo Alto","Utuado","Vega Alta","Vega Baja","Villalba","Yabucoa","Yauco","Vieques Municipality"


How much coverage do we get through this matching alone?

In [55]:
def tags_to_txt(x):
    return ' '.join([y[0] for y in x])
PR_state_names = ['PR ', 'Puerto Rico ']
PR_state_names = [clean_txt(x) for x in PR_state_names]
data_valid = data_valid.assign(**{'has_state_descriptor' : data_valid.apply(lambda x: match_city_loc_pattern(x.loc['NE'], tags_to_txt(x.loc['status_message_tags']), PR_state_names), axis=1)})
data_valid = data_valid.assign(**{'has_muni_descriptor' : data_valid.apply(lambda x: match_city_loc_pattern(x.loc['NE'], x.loc['status_message'], PR_muni_names), axis=1)})

In [56]:
print('%d/%d NEs with muni descriptor'%(data_valid.loc[:, 'has_muni_descriptor'].sum(), data_valid.shape[0]))
print('%d/%d NEs with state descriptor'%(data_valid.loc[:, 'has_state_descriptor'].sum(), data_valid.shape[0]))

353/9389 NEs with muni descriptor
72/9389 NEs with state descriptor


In [57]:
data_valid[data_valid.loc[:, 'has_state_descriptor']].loc[:, ['status_message', 'NE']].head(10).values

array([['Mi gente tenemos que ayudar a florecer a nuestra Isla Bella. Dejemos de criticar y actuemos unamonos, para levantar a nuestra Isla, Adopten a una Familia en Puerto Rico y envíen una caja de comida todos los meses. Mi gente de Puerto Rico vamos a ayudarlos. Mi gente de Corozal estamos aquí, nos hemos unidos en Zellow en el Canal Corozal PR los invitos juntos a la pagina en Facebook Corozal Huracán Maria a que trabajemos por Puerto Rico. Nosotros desde los Estados Unidos podemos hacer la diferencia. Mi gente hermosa de Puerto Rico y Corozal estamos con ustedes. #Puertoricoselevanta',
        'Corozal'],
       ['Mi gente tenemos que ayudar a florecer a nuestra Isla Bella. Dejemos de criticar y actuemos unamonos, para levantar a nuestra Isla, Adopten a una Familia en Puerto Rico y envíen una caja de comida todos los meses. Mi gente de Puerto Rico vamos a ayudarlos. Mi gente de Corozal estamos aquí, nos hemos unidos en Zellow en el Canal Corozal PR los invitos juntos a la pagina e

In [60]:
display(data_valid.loc[:, ['NE', 'status_message', 'status_message_tags',]].iloc[550:580, :].values)

array([['Barrio Nuevo',
        'Hola.. Estamos buscando noticias sobre el pueblo Barrio Nuevo.. sector Bunker Hill. Mi tío Primitivo Rivera vive ahí y hasta la hora no hemos podido comunicarnos con el. El tiene 80 años. Es diabético y su esposa está enferma. Ya han pasado 10 días sin oír si está bien.. si tiene comida y sus medicina. Les suplico, si alguien sabe algo...',
        list([('Hola', 'O'), ('.', 'O'), ('.', 'O'), ('Estamos', 'O'), ('buscando', 'O'), ('noticias', 'O'), ('sobre', 'O'), ('el', 'O'), ('pueblo', 'O'), ('Barrio', 'LOCATION'), ('Nuevo', 'LOCATION'), ('.', 'O'), ('.', 'O'), ('sector', 'O'), ('Bunker', 'PERSON'), ('Hill', 'PERSON'), ('.', 'O'), ('Mi', 'O'), ('tío', 'O'), ('Primitivo', 'ORGANIZATION'), ('Rivera', 'ORGANIZATION'), ('vive', 'O'), ('ahí', 'O'), ('y', 'O'), ('hasta', 'O'), ('la', 'O'), ('hora', 'O'), ('no', 'O'), ('hemos', 'O'), ('podido', 'O'), ('comunicar', 'O'), ('nos', 'O'), ('con', 'O'), ('el', 'O'), ('.', 'O'), ('El', 'O'), ('tiene', 'O'), ('80', '

## Test parses
Let's see how much our coverage improves if we use the parse tree.

First we have to assign valid importance stats.

In [67]:
import pickle
## add importance vars
geonames_data = pickle.load(open('/hg190/corpora/GeoNames/allCountriesSimplified_lookup_US.pickle', 'rb'))
# restrict to PR
geonames_data_PR = {k : v[v.loc[:, 'country']=='PR'] for k,v in geonames_data.items()}
geonames_data_PR = {k : v for k,v in geonames_data_PR.items() if v.shape[0] > 0}
geonames_PR_max_pop = {k : v.loc[:, 'population'].max() for k,v in geonames_data_PR.items()}
geonames_PR_max_alt_names = {k : v.loc[:, 'alternate_name_count'].max() for k,v in geonames_data_PR.items()}

In [70]:
data_valid = data_valid.assign(**{
    'NE_clean' : data_valid.loc[:, 'NE'].apply(lambda x: unidecode(str(x).replace('_', ' ').lower()))
})
data_valid = data_valid.assign(**{
    'max_population' : data_valid.loc[:, 'NE_clean'].apply(lambda x: geonames_PR_max_pop[x] if x in geonames_PR_max_pop else 0.),
    'max_alternate_names' : data_valid.loc[:, 'NE_clean'].apply(lambda x: geonames_PR_max_alt_names[x] if x in geonames_PR_max_alt_names else 0.),
})

In [71]:
data_valid.head()

,group_id,status_author_id,status_message,status_lang,status_id,status_published,status_message_clean,status_message_tags,status_message_tags_ne,status_len,status_message_tags_ne_LOC,LOC_GN,LOC_OSM,NE,valid_loc,has_state_descriptor,has_muni_descriptor,NE_clean,max_population,max_alternate_names
0,351272391991842,10159623474155515,Estoy preocupada por mi tia vicky vazquez que ...,es,351272391991842_362333787552369,2017-10-19 18:25:23,Estoy preocupada por mi tia vicky vazquez que ...,"[(Estoy, O), (preocupada, O), (por, O), (mi, O...","[('asma', 'CAUSE_OF_DEATH'), ('Dorado', 'ORGAN...",579,['Dorado'],1,1,Dorado,1,False,False,dorado,38165.0,8.0
2,351272391991842,141039043205101,Pueblo de Puerto Rico no se monten en las gran...,es,351272391991842_358929311226150,2017-10-11 14:14:22,Pueblo de Puerto Rico no se monten en las gran...,"[(Pueblo, O), (de, O), (Puerto, COUNTRY), (Ric...","[('Puerto Rico', 'COUNTRY'), ('America', 'MISC...",356,['shelter'],0,1,shelter,1,False,False,shelter,0.0,0.0
3,351272391991842,2060154724214607,"Guayama, Dorado, y Piñones",es,351272391991842_357797888005959,2017-10-08 14:52:49,"Guayama, Dorado, y Piñones","[(Guayama, LOCATION), (,, O), (Dorado, O), (,,...","[('Guayama', 'LOCATION')]",26,['Guayama'],1,1,Guayama,1,False,True,guayama,45362.0,11.0
4,351272391991842,10155779649437154,"Estoy buscando mi tío, Ivan J. Porrata. Su her...",es,351272391991842_355121381606943,2017-10-01 21:46:29,"Estoy buscando mi tío, Ivan J. Porrata. Su her...","[(Estoy, O), (buscando, O), (mi, O), (tío, O),...","[('Ivan J. Porrata', 'PERSON'), ('Tulio', 'PER...",309,"['Utuado', 'Toa', 'Baja', 'P.O. Box', 'Dorado'...",1,1,Utuado,1,False,False,utuado,33149.0,7.0
8,351272391991842,10155779649437154,"Estoy buscando mi tío, Ivan J. Porrata. Su her...",es,351272391991842_355121381606943,2017-10-01 21:46:29,"Estoy buscando mi tío, Ivan J. Porrata. Su her...","[(Estoy, O), (buscando, O), (mi, O), (tío, O),...","[('Ivan J. Porrata', 'PERSON'), ('Tulio', 'PER...",309,"['Utuado', 'Toa', 'Baja', 'P.O. Box', 'Dorado'...",1,1,Dorado,1,False,False,dorado,38165.0,8.0


In [164]:
from ast import literal_eval
import re
## load parse data
parse_matcher = re.compile('(.+)/(.+)/(.+)/(.+)')
def str_to_parse(x):
    x_list = [y for y in x.split(' ') if y.strip() != '']
#     for y in x_list:
#         try:
#             parse_matcher.search(y).group(0)
#         except Exception as e:
#             print('raw %s'%(x))
#             print('error = %s in %s'%(y, str(x_list)))
    x_list = [y for y in x_list if parse_matcher.search(y) is not None]
    x_parse = [parse_matcher.search(y).group(0).split('/') for y in x_list]
    x_parse = [y for y in x_parse if len(y)==4]
#     for y in x_parse:
#         try:
#             [y[0], y[1], int(y[2]), y[3]]
#         except Exception as e:
#             print('error = %s'%(str(x_parse)))
    x_parse = [[y[0], y[1], int(y[2]), y[3], i] for i, y in enumerate(x_parse)]
    return x_parse
data_parsed = pd.read_csv('../../data/facebook-maria/combined_group_data_es_tagged_parsed.txt', sep='\t', index_col=False, header=None)
data_parsed.columns = ['status_id', 'parse']
data_parsed = data_parsed.assign(**{'parse' : data_parsed.loc[:, 'parse'].apply(lambda x: str_to_parse(x))})
# stack parses
data_parsed = pd.concat([pd.Series([i, [y for y in x.loc[:, 'parse'].values if len(y) > 0]], index=['status_id', 'parse']) for i,x in data_parsed.groupby('status_id')], axis=1).transpose()
display(data_parsed.head())

,status_id,parse
0,105121436871597_105122476871493,"[[[Lo, DET, 1, det, 0], [que, PRON, 3, dobj, 1..."
1,105121436871597_105140713536336,"[[[Mi, PRON, 1, det, 0], [gente, NOUN, 2, attr..."
2,105121436871597_105141490202925,"[[[Artículos, NOUN, 0, root, 0], [que, PRON, 4..."
3,105121436871597_105161620200912,"[[[Por, ADP, 3, discourse, 0], [favor, NOUN, 0..."
4,105121436871597_106853466698394,"[[[Quisiera, VERB, 0, root, 0], [saber, VERB, ..."


In [167]:
data_valid_parse = pd.merge(data_valid, data_parsed, on='status_id', how='inner')
print('%d/%d data'%(data_valid_parse.shape[0], data_valid.shape[0]))
## add data name for anchor detection
data_valid_parse = data_valid_parse.assign(**{'data_name' : 'maria_fb'})

9238/9389 data


In [166]:
for i, data_i in data_valid_parse.iterrows():
    for p in data_i.loc['parse']:
        if(len(p) == 0):
            print('error with data %s'%(data_i))

In [173]:
data_valid_parse.columns

Index(['group_id', 'status_author_id', 'status_message', 'status_lang',
       'status_id', 'status_published', 'status_message_clean',
       'status_message_tags', 'status_message_tags_ne', 'status_len',
       'status_message_tags_ne_LOC', 'LOC_GN', 'LOC_OSM', 'NE', 'valid_loc',
       'has_state_descriptor', 'has_muni_descriptor', 'NE_clean',
       'max_population', 'max_alternate_names', 'parse', 'data_name'],
      dtype='object')

In [176]:
a = list(data_valid_parse.groupby(id_var))
print(a[0][1])

           group_id   status_author_id  \
28  105121436871597  10155826475814621   

                                       status_message status_lang  \
28  Lo que tengo entendido es que el puente de la ...          es   

                          status_id     status_published  \
28  105121436871597_105122476871493  2017-09-21 11:27:50   

                                 status_message_clean  \
28  Lo que tengo entendido es que el puente de la ...   

                                  status_message_tags  \
28  [(Lo, O), (que, O), (tengo, O), (entendido, O)...   

           status_message_tags_ne  status_len  ... LOC_OSM       NE  \
28  [('Maunabo', 'ORGANIZATION')]         130  ...       1  Maunabo   

    valid_loc has_state_descriptor  has_muni_descriptor  NE_clean  \
28          1                False                False   maunabo   

    max_population max_alternate_names  \
28         12225.0                 1.0   

                                                parse  dat

In [188]:
from importlib import reload
import data_helpers
reload(data_helpers)
from data_helpers import detect_anchor_by_type
def detect_anchor(data, anchor_types_to_use=['state', 'descriptor']):
#     print('processing %d data'%(data.shape[0]))
    NE_var = 'NE'
    data_name_var = 'data_name'
    anchor_var = 'max_population'
    txt_var = 'status_message'
    anchor_types_all = ['state', 'descriptor', 'anchor_compound', 'anchor_list']
    anchor_state, anchor_descriptor, anchor_compound, anchor_list = detect_anchor_by_type(data, NE_var=NE_var, data_name_var=data_name_var, anchor_var=anchor_var, txt_var=txt_var)
#     print(data.loc[:, NE_var])
#     print(anchors)
#     print('anchor state %s'%(anchor_state))
    anchors = pd.concat([anchor_state, anchor_descriptor, anchor_compound, anchor_list], axis=1)
#     print(anchors)
    anchors.columns = anchor_types_all
#     print('final anchors %s'%(str(anchors)))
    anchors = anchors.loc[:, anchor_types_to_use]
    anchors = anchors.max(axis=1)
    data = data.assign(**{'anchor' : anchors.values})
    return data
id_var = 'status_id'
anchor_types_to_use = ['state', 'descriptor']
# test_data = data_valid_parse[data_valid_parse.loc[:, id_var] == data_valid_parse.loc[:, id_var].iloc[10]]
# print(detect_anchor(test_data, anchor_types_to_use=anchor_types_to_use))
anchor_data = []
for id_i, data_i in data_valid_parse.groupby(id_var):
    anchor_data_i = detect_anchor(data_i, anchor_types_to_use=anchor_types_to_use)
    anchor_data.append(anchor_data_i)
anchor_data = pd.concat(anchor_data, axis=0)
# data_valid_parse_anchor = pd.concat([x[1] for x in data_valid_parse.groupby(id_var).apply(lambda x: )], axis=0)

In [180]:
print('%d/%d anchors'%(anchor_data.loc[:, 'anchor'].sum(), anchor_data.shape[0]))

583/9238 anchors


Let's see what these anchors caught.

In [184]:
anchor_data[anchor_data.loc[:, 'anchor']==1].head(10).loc[:, ['NE', 'status_message']].values

array([['Manati',
        'Si alguien conoce a estas personas por favor déjennos saber la familia en Texas está preocupada(Magdalena) Por favor compártanlo para ver si logramos saber de ellos.Oskar Jimenez, Crr. 668 Cordova Davilla, Bz. 188 Manati, PR 00674'],
       ['Buena Vista',
        "This is my niece's Grandfather Manuel Mendoza. We still haven’t heard from him. He lives in Buena Vista Humacao. We just want to know you’re okay ❤️ We will continue praying for your safety. Este es el abuelo de mis sobrinas,  Manuel Mendoza. Aún no hemos oído hablar de él. Vive en Buena Vista Humacao. Sólo queremos saber que estás bien ❤️ Seguiremos orando por tu seguridad."],
       ['Buena Vista',
        "This is my niece's Grandfather Manuel Mendoza. We still haven’t heard from him. He lives in Buena Vista Humacao. We just want to know you’re okay ❤️ We will continue praying for your safety. Este es el abuelo de mis sobrinas,  Manuel Mendoza. Aún no hemos oído hablar de él. Vive en Buena Vista

These look legit! Mostly CITY, LOC patterns but also a few descriptor phrases throughout.

## Check anchor coverage in groups
Make sure that we have sufficient coverage across groups.

In [186]:
group_var = 'group_id'
display(anchor_data.groupby(group_var).apply(lambda x: x.loc[:, 'anchor'].sum()))

group_id
105121436871597       0
117224992282301       3
117445378946554      25
117529462276443       0
118787682128186      13
119087272129297      28
119132632075224       9
119987445374151       0
127217134598253      33
128909594424200       0
130010134396359       1
130583424251881       2
130913387550000      21
132022090761950      13
132392547395104      28
132963337341347      27
142680509678003      13
185336475344755       6
210130182858104       2
228066954387757      10
239379013253824      23
239708553223790       1
301495306993376       0
311739902631820       3
320205998443003      10
351272391991842       3
352124068563860       0
477086339329141       1
486819048360070       8
505778186467207       2
721270821393500      10
727610640755618       2
782660231920150       1
803659183149847       5
844368579056187       2
871880559633183       0
889808494515226       1
891721064308258      48
986662628142079       4
1103661653103604      1
1201507393289356      0
1306772

OK! Not great coverage but definitely better than nothing.